In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
from sklearn import tree
import pandas as pd
import os

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec','koi_period','koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_slogg','koi_kepmag', 'koi_impact', 'dec']]
selected_features.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_duration  koi_depth  koi_prad  koi_teq  koi_insol  \
0   54.418383       4.50700      874.8      2.83      443       9.11   
1   19.899140       1.78220    10829.0     14.60      638      39.30   
2    1.736952       2.40641     8079.2     33.46     1395     891.96   
3    2.525592       1.65450      603.3      2.75     1406     926.16   
4    4.134435       3.14020      686.0      2.77     1160     427.65   

   koi_slogg  koi_kepmag  koi_impact        dec  
0      4.467      15.347       0.586  48.141651  
1      4.544      15.436       0.969  48.134129  
2      4.564      15.597       1.276  48.285210  
3      4.438      15.509       0.701  48.226200  
4      4.486      15.714       0.762  48.224670

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
y = selected_features['koi_disposition']
X = selected_features[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec','koi_period','koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_slogg','koi_kepmag', 'koi_impact', 'dec']]
print(X.shape, y.shape)

(6991, 14) (6991,)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Scale Data and Set up Decision Tree

Scale the data using the MinMaxScaler

In [8]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Set up Decision Tree Model & print train/test scores
clf = tree.DecisionTreeClassifier(max_depth=12)
clf = clf.fit(X_train_scaled, y_train)
print('Train score: {}'.format(clf.score(X_train_scaled, y_train)))
print('Test score: {}'.format(clf.score(X_test_scaled, y_test)))

Train score: 0.9189395384321953
Test score: 0.8506864988558352


# Set up Random Forest to Score Model



In [18]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=40)
rf = rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)

0.8764302059496567

# Use Random Forest to Determine Feature Importance

In [19]:
feature_names = X.columns

In [20]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.13550549500500014, 'koi_fpflag_nt'),
 (0.12146507249341613, 'koi_fpflag_co'),
 (0.10624934748449828, 'koi_fpflag_ss'),
 (0.10165143624424937, 'koi_depth'),
 (0.09233239591108945, 'koi_prad'),
 (0.07428831890449272, 'koi_period'),
 (0.054007292095832216, 'koi_impact'),
 (0.05012821867176513, 'koi_teq'),
 (0.04948986810220175, 'koi_duration'),
 (0.048756305436351004, 'koi_kepmag'),
 (0.04412586526135952, 'koi_insol'),
 (0.04402512017500011, 'koi_slogg'),
 (0.043636515512540545, 'koi_fpflag_ec'),
 (0.03433874870220366, 'dec')]

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [25]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [100, 200, 300],
              'min_samples_leaf': [1, 2, 3]}
grid = GridSearchCV(RandomForestClassifier(), param_grid, verbose=3)

In [26]:
# Train the model with GridSearch
# Fit the model using the grid search estimator. 
grid.fit(X_train_scaled, y_train)

C:\Users\siris\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] min_samples_leaf=1, n_estimators=100 ............................
[CV]  min_samples_leaf=1, n_estimators=100, score=0.878, total=   1.3s
[CV] min_samples_leaf=1, n_estimators=100 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV]  min_samples_leaf=1, n_estimators=100, score=0.864, total=   1.3s
[CV] min_samples_leaf=1, n_estimators=100 ............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.5s remaining:    0.0s


[CV]  min_samples_leaf=1, n_estimators=100, score=0.868, total=   1.3s
[CV] min_samples_leaf=1, n_estimators=200 ............................
[CV]  min_samples_leaf=1, n_estimators=200, score=0.879, total=   2.6s
[CV] min_samples_leaf=1, n_estimators=200 ............................
[CV]  min_samples_leaf=1, n_estimators=200, score=0.868, total=   2.6s
[CV] min_samples_leaf=1, n_estimators=200 ............................
[CV]  min_samples_leaf=1, n_estimators=200, score=0.872, total=   2.5s
[CV] min_samples_leaf=1, n_estimators=300 ............................
[CV]  min_samples_leaf=1, n_estimators=300, score=0.881, total=   3.9s
[CV] min_samples_leaf=1, n_estimators=300 ............................
[CV]  min_samples_leaf=1, n_estimators=300, score=0.868, total=   3.9s
[CV] min_samples_leaf=1, n_estimators=300 ............................
[CV]  min_samples_leaf=1, n_estimators=300, score=0.872, total=   3.9s
[CV] min_samples_leaf=2, n_estimators=100 ............................
[CV]  

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.1min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
           

In [27]:
# Determine best parameter and best score
print(grid.best_params_)
print(grid.best_score_)

{'min_samples_leaf': 3, 'n_estimators': 100}
0.8743086019454511


# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'Decision_Tree.sav'
joblib.dump(your_model, filename)